In [2]:
from hyperopt import hp, fmin, tpe, space_eval
from hyperopt.pyll.stochastic import sample
from math import log

In [43]:
# Old-net configuration
min_layers = 1
max_layers = 3

# sets up the neural network
nnets = [None] * (max_layers - min_layers + 1)

for i, num_layers in enumerate(range(min_layers, max_layers + 1)):
    num_hids = [None] * num_layers
    for j in range(num_layers):
        num_hids[j] = hp.qloguniform(
            'num_hid_%i%i' % (i, j), log(10), log(100), 1)

    nnets[i] = num_hids

# define the hyperparamater space to search
hyperspace = {'mln_params': [
    {'arch': hp.choice('arch', nnets)},           
    {'l1_reg': hp.choice(
        'l1_lambda', [None, hp.loguniform('l1_decay', log(1e-5), log(10))])},
    {'l2_reg': hp.choice(
        'l2_lambda', [None, hp.loguniform('l2_decay', log(1e-5), log(10))])},
],
    'optim_params': [
    {'learn_rate': hp.uniform('learn_rate', 0, 1)},
    {'rho': hp.uniform('rho', 0, 1)},
    {'num_epochs': hp.qloguniform(
        'num_epochs', log(10), log(1e4), 1)},
    {'batch_size': hp.quniform('batch_size', 128, 1024, 1)},
    {'init_method': hp.choice(
        'init_method', ['gauss', 'fan-io'])},
    {'scale_factor': hp.uniform(
        'scale_factor', 0, 1)}
]
}
sample(hyperspace)

{'mln_params': ({'arch': (19.0,)},
  {'l1_reg': 0.0022512814271359012},
  {'l2_reg': 1.0362276138845081}),
 'optim_params': ({'learn_rate': 0.8673858561486274},
  {'rho': 0.6542634453039271},
  {'num_epochs': 22.0},
  {'batch_size': 442.0},
  {'init_method': 'fan-io'},
  {'scale_factor': 0.8317983892400194})}

In [62]:
# Force at least 2 layers, cuz we're modern
min_layers = 2
max_layers = 3

# sets up the neural network
nnets = [None] * (max_layers - min_layers + 1)

for i, num_layers in enumerate(range(min_layers, max_layers + 1)):
    num_hids = [None] * num_layers
    for j in range(num_layers):
        num_hids[j] = hp.qloguniform(
            'num_hid_%i%i' % (i, j), log(100), log(1000), 1)

    nnets[i] = num_hids

# define the hyperparamater space to search
hyperspace = ({'mln_params': [
    {'arch': hp.choice('arch', nnets)},
    {'input_p': hp.uniform('ip', 0, 1)},
    {'hidden_p': hp.uniform('hp', 0, 1)},
    {'l1_reg': hp.choice(
        'l1_lambda', [None, hp.loguniform('l1_decay', log(1e-5), log(10))])},
    {'l2_reg': hp.choice(
        'l2_lambda', [None, hp.loguniform('l2_decay', log(1e-5), log(10))])},
],
    'optim_params': [
    {'learn_rate': hp.uniform('learn_rate', 0, 1)},
    {'rho': hp.uniform('rho', 0, 1)},
    {'num_epochs': hp.qloguniform(
        'num_epochs', log(1e2), log(2000), 1)},
    {'batch_size': hp.quniform('batch_size', 128, 1024, 1)},
    {'scale_factor': hp.uniform(
        'scale_factor', 0, 1)}
]
})
sample(hyperspace)

{'mln_params': ({'arch': (812.0, 234.0)},
  {'input_p': 0.882908327943932},
  {'hidden_p': 0.047493794381396115},
  {'l1_reg': None},
  {'l2_reg': 0.009801015366450113}),
 'optim_params': ({'learn_rate': 0.9864581984539547},
  {'rho': 0.26708194578840005},
  {'num_epochs': 865.0},
  {'batch_size': 828.0},
  {'scale_factor': 0.5838919196352292})}

In [90]:
hyperspace = ({'x_value':hp.uniform('x',-1,1),'y_value':hp.choice('y', [0.0001,hp.uniform('this_y',-1,1)])})

def objective_fn(space):
    x = space['x_value']
    y = space['y_value']
    return x**2 + y**2

best = fmin(objective_fn,hyperspace,algo=tpe.suggest,max_evals=100)
print best

{'y': 0, 'x': 0.004251807848178524}


In [5]:
# proof that the 'space' you pass into the objective function can be a combination of default and learnable parameters
hyperspace_v1 = {'x':hp.uniform('x',-1,1),'y':hp.uniform('y',-1,1)}
hyperspace_v2 = {'x':hp.uniform('x',-1,1),'y':hp.uniform('y',-1,1), 'z':5}

def objective_fn_v1(space):
    x = space['x']
    y = space['y']
    return x**2 + y**2

def objective_fn_v2(space):
    x = space['x']
    y = space['y']
    z = space['z']
    return x**2 + y**2 + z

best_v1 = fmin(objective_fn_v1,hyperspace_v1,algo=tpe.suggest,max_evals=100)
best_v2 = fmin(objective_fn_v2,hyperspace_v2,algo=tpe.suggest,max_evals=100)
print best_v1
print best_v2

{'y': -0.04528889837688501, 'x': 0.016277551583606824}
{'y': -0.04528889837688501, 'x': 0.016277551583606824}
